In [2]:
from keras import backend as Keras
from keras.backend.tensorflow_backend import set_session
import multiprocessing
import numpy as np
import os
import pickle
import sys
import tensorflow as tf

sys.path.insert(0, "../../Code/nn_breaking_detection")
from setup_mnist import MNISTModel, MNIST
from setup_cifar import CIFARModel, CIFAR

sys.path.insert(0, "../../Code/cleverhans")
from cleverhans.model import Model
from cleverhans.attacks import SPSA

sys.path.insert(0, "../../Code/")
from defense import DefendedModel
from nearest_neighbors import knn

%matplotlib inline
import matplotlib.pyplot as plt

from scipy.stats import mode

Using TensorFlow backend.


In [8]:
CIFAR_data = CIFAR()
MNIST_data = MNIST()

## 1. Retrieving the Data and Loading the Models

In [9]:
pickle_in = open("../5-Attack/CIFAR/0.031_1_51_0.0","rb")
attack_dict = pickle.load(pickle_in)
fp_indices = np.where(attack_dict['FP'] == True)
fp_test_x = CIFAR_data.test_data[fp_indices]
fp_test_y = np.argmax(CIFAR_data.test_labels[fp_indices], axis = 1)

NameError: name 'CIFAR_data' is not defined

## 2. Performing Projection

In [10]:
# Numpy Implementation
def random_peturb(image, attempts=200, dist_type="gaussian", params = {"std_dev": 0.02}):
    size = (attempts, ) + image.shape

    if dist_type == "gaussian": 
        deltas = np.random.normal(0, params["std_dev"], size)
    elif dist_type == "uniform":
        deltas = np.random.uniform(params["low"], params["high"], size)    
    else: 
        print("Noise type not supported")
        
    projection_attempts = image + deltas
    return projection_attempts


    

In [11]:
projection_attempts = random_peturb(fp_test_x[0], attempts=10)
projection_attempts

NameError: name 'fp_test_x' is not defined

## 3. Testing if Projection Worked

In [12]:
### SETTING UP TENSORFLOW MODEL ###
Keras.clear_session()

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

sess = Keras.get_session()
Keras.set_learning_phase(False)

# Fix Random Seeds
np.random.seed(1)
tf.set_random_seed(1) #Having this before keras.clear_session() causes it it hang for some reason
base_model = CIFARModel("../1-Models/CIFAR")

x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3))
K = 51

dataset = "CIFAR"
mode = "1"
x_train_real = np.squeeze(np.load("../3-Representation/" + dataset + "/train_" + mode + ".npy"))
x_train_adv = np.squeeze(np.load("../3-Representation/" + dataset + "/train_adv_" + mode + ".npy"))

n_train = x_train_real.shape[0]
n_train_adv = x_train_adv.shape[0]
x_train = np.float32(np.vstack((x_train_real, x_train_adv)))
y_train = np.float32(np.hstack((-1.0 * np.ones(n_train), np.ones(n_train_adv))))

# Create the defended model
model_defended = DefendedModel(base_model, x_train, y_train, K)
defended_logits = model_defended.get_logits(x)
#logits_real = sess.run(defended_logits, {x: projection_attempts})




In [23]:
### PROJECTION PARAMS ###
attempts = 1000
dist_type = "gaussian"
projections = 0
successes = 0
gaussian_params = {"std_dev": 0.022}
uniform_params = {'low': -0.2, 'high': 0.2}


### PROJECTION ###
for i in range(fp_test_x.shape[0]):
    projection_attempts = random_peturb(fp_test_x[i], attempts, dist_type, gaussian_params)
    projection_labels = np.argmax(sess.run(defended_logits, {x: projection_attempts}), axis = 1)
    successful_projections = projection_labels[np.nonzero(projection_labels * (projection_labels != 10))]

    
    if successful_projections.shape[0] != 0:
        projections += 1
    
        if mode(successful_projections)[0][0] == fp_test_y[i]:
            successes += 1

    if i % 25 == 0:
        print("Done with image ", i)
        
print("\nPercent Projected: ", projections/fp_test_x.shape[0])
print("Percent Projected Successfully: ", successes/fp_test_x.shape[0])
print("Reduction of FPR: ", successes/1000)
print("Total Percent Predicted Correctly: ", 1 - 0.229 + successes/1000)

Done with image  0
Done with image  25
Done with image  50
Done with image  75
Done with image  100
Done with image  125
Done with image  150
Done with image  175
Done with image  200
Done with image  225

Percent Projected:  0.8864628820960698
Percent Projected Successfully:  0.49344978165938863
Reduction of FPR:  0.113
Total Percent Predicted Correctly:  0.884


In [15]:
fp_indices[0].shape

(229,)

In [13]:
import defense_projection
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3))
defense = defense_projection.DefendedModel(base_model, x_train, y_train, K)
get_votes = defense.get_logits(x) 

In [15]:
sess.run(get_votes, {x: fp_test_x})

0.92156863

In [15]:
x = tf.placeholder(tf.float32, shape=(None,2,3))
y = x[0][0][0] + 1

In [16]:
sess.run(y, {x:np.linspace(1,6,6).reshape((1,2,3))})

2.0

In [17]:
delta = tf.random_normal(
    (1,2,3),
    mean=0.0,
    stddev=1.0,
    dtype=tf.float32,
)

y = delta + x

In [18]:
sess.run(y, {x: np.linspace(1,12,12).reshape((2,2,3))})

array([[[ 1.2399081,  1.9938997,  2.7676735],
        [ 6.0171127,  5.0995693,  4.9201517]],

       [[ 7.239908 ,  7.9939   ,  8.7676735],
        [12.017113 , 11.099569 , 10.920152 ]]], dtype=float32)

In [19]:
delta_2 = tf.random_normal(delta.shape)

In [20]:
sess.run(delta_2)

array([[[ 1.3225639 , -2.5372453 , -0.5415378 ],
        [ 0.11667927, -0.21633413, -0.3490997 ]]], dtype=float32)

In [21]:
sess.run(delta)

array([[[-0.28913534, -1.6328075 , -0.50871074],
        [ 0.01319283,  0.0617231 , -1.238702  ]]], dtype=float32)

In [22]:
delta_2.shape[1:]

TensorShape([Dimension(2), Dimension(3)])